In [60]:
import nltk
# nltk.download('punkt')
import numpy as np
from nltk.tokenize import sent_tokenize
import json
import itertools
import re
import os
from collections import defaultdict

In [22]:
# base_path = "../../data/test/"
# file_name = "164726"
base_path = "../../../train/"
file_name = "100187"

In [25]:
all_files = os.listdir(base_path)
filenames = set([name.split(".")[0] for name in all_files])

In [3]:
ann_str = None
with open(base_path + file_name + ".ann", "r") as file:
    ann_str = file.read()

In [39]:
def parse_entities_relationships(file_content):
    """
    parses out entities and relationships from the entities file
    """
    entities = {}
    relationships = {}

    for line in file_content.strip().split('\n'):
        # madarchod line might be annotator comment
        if line.strip().startswith("#"):
            continue
        cols = line.split('\t')
        identifier = cols[0]
        if "Arg1:" in cols[1]:
            relationship_type, arg1, arg2 = cols[1].split()
            relationships[identifier] = {
                'type': relationship_type,
                'arg1': arg1.split(':')[1],
                'arg2': arg2.split(':')[1]
            }
            
        else:
            split_cols = cols[1].split()
            entity_type = split_cols[0]
            value = cols[2]
            start = split_cols[1]
            end = split_cols[-1]
            entities[identifier] = {
                'type': entity_type,
                'start': int(start),
                'end': int(end),
                'value': value
            }

    return entities, relationships

In [40]:
txt_str = None
with open(base_path + file_name + ".txt", "r") as file:
    txt_str = file.read()

In [41]:
ent, rel = parse_entities_relationships(ann_str)

In [82]:
def get_spans(entities, relationships, txt_str, split_char="\n"):
    """
    first pass through spans
    """
    spans = {}
    for rel in relationships.keys():
        arg1 = entities[relationships[rel]['arg1']]
        arg2 = entities[relationships[rel]['arg2']]
        min_left = min(arg1['start'], arg2['start'])
        max_right = max(arg1['end'], arg2['end'])
        # find dots
        prev_char = txt_str[:min_left].rfind(split_char)
        next_char = txt_str[max_right:].find(split_char)
        # see if no exist
        assert(prev_char != -1)
        assert(next_char != -1)
        # see that span is not insanely large
        assert(next_char-prev_char <= 2*(max_right-min_left))
        # return spans
        spans[rel] = {
            'start':prev_char,
            'end':next_char
        }
    return spans

In [83]:
def get_ents_in_spans(spans, ents):
    span_ent = defaultdict(list)
    for span in spans.keys():
        s = spans[span]
        for ent in ents.keys():
            e = ents[ent]
            if s['start'] <= e['start'] and s['end'] >= e['end']:
                span_ent[s].append(e)
    return span_ent

In [84]:
# CHECKING FOR ALL FILES MA CHUDAO
for file_name in filenames:
    print(file_name)
    # read annotations
    ann_str = None
    with open(base_path + file_name + ".ann", "r") as file:
        ann_str = file.read()
    # parse entities and relations
    ent, rel = parse_entities_relationships(ann_str)
    # parse text string
    txt_str = None
    with open(base_path + file_name + ".txt", "r") as file:
        txt_str = file.read()
    # first pass through spans
    spans = get_spans(ent, rel, txt_str, "\n")
    # second pass through spans
    span_ents = get_ents_in_spans(spans, ent)

186788
109450
102324
120237
161477
101857
112615
109566
109697
105106
155980
109770
106939
107322
185759
188551
124307
103926
113705
112140
102365
104979
125867
103315
102698
157352
115789
152612
159079
108809
100229
120301
197869
186134
127263
126380
108032
151688
103639
143562
110863
110499
106629
105445
106026
187852
103377
111923
117609
111323
106038
182160
102045
127741
118418
112249
107869
101779
102456
191016
184696
100922
130153
128041
110186
195784
108901
110727
160574
101665
113391
100579
123475
102019
144301
105747
115021
113813
115232
157609
110326
100035
195689
102027
107872
117872
124891
124917
182003
104468
107139
161384
102053
119573
113824
105954
191162
105050
108073
104095
168936
101427
106423
107058
100883
114959
109191
116853
130076
115191
114248
106575
130440
104091
109866
174150
111458
123244
102527
117156
190444
114965
101092
134445
114680
113524
103677
109617
168915
119069
149687
189471
104682
192877
123589
103722
101372
114144
112747
103317
108754
114836
101215